In [1]:
import pandas as pd
import psycopg2

In [54]:
# Устанавливаем соединение с базой данных
conn = psycopg2.connect(user="postgres", 
                        password="1234554321live##Test", 
                        host="localhost", 
                        database="postgres")

In [55]:
# Создаем объект курсора
cur = conn.cursor()

1) Вывести распределение (количество) клиентов по сферам деятельности, отсортировав результат по убыванию количества. — (1 балл)
2) Найти сумму транзакций за каждый месяц по сферам деятельности, отсортировав по месяцам и по сфере деятельности. — (1 балл)
3) Вывести количество онлайн-заказов для всех брендов в рамках подтвержденных заказов клиентов из сферы IT. — (1 балл)
4) Найти по всем клиентам сумму всех транзакций (list_price), максимум, минимум и количество транзакций, отсортировав результат по убыванию суммы транзакций и количества клиентов. Выполните двумя способами: используя только group by и используя только оконные функции. Сравните результат. — (2 балла)
5) Найти имена и фамилии клиентов с минимальной/максимальной суммой транзакций за весь период (сумма транзакций не может быть null). Напишите отдельные запросы для минимальной и максимальной суммы. — (2 балла)
6) Вывести только самые первые транзакции клиентов. Решить с помощью оконных функций. — (1 балл)
7) Вывести имена, фамилии и профессии клиентов, между транзакциями которых был максимальный интервал (интервал вычисляется в днях) — (2 балла).

1) Вывести распределение (количество) клиентов по сферам деятельности, отсортировав результат по убыванию количества. — (1 балл)

In [15]:
# Выполняем SQL-запрос
cur.execute("""
            select  job_industry_category,
                    count(customer_id) as counts
            from homework_2.customer as c
            group by 1 
            order by 2 desc
            """)
conn.commit()
results = cur.fetchall()
# Получаем описание столбцов таблицы
columns = [desc[0] for desc in cur.description]
# Создаем датафрейм с нужными названиями столбцов
industry_counts = pd.DataFrame(results, columns=columns)
industry_counts

,job_industry_category,counts
0,Manufacturing,799
1,Financial Services,774
2,NaN,656
3,Health,602
4,Retail,358
5,Property,267
6,IT,223
7,Entertainment,136
8,Argiculture,113
9,Telecommunications,72


2) Найти сумму транзакций за каждый месяц по сферам деятельности, отсортировав по месяцам и по сфере деятельности. — (1 балл)

In [24]:
# Выполняем SQL-запрос
cur.execute("""
            select  t.transaction_date::date as date,
                    c.job_industry_category,
                    sum(t.list_price) as sum
            from homework_2.transaction as t
            left join homework_2.customer as c
            on t.customer_id = c.customer_id
            group by 1, 2
            order by 1, 2
            """)
conn.commit()
results = cur.fetchall()
# Получаем описание столбцов таблицы
columns = [desc[0] for desc in cur.description]
# Создаем датафрейм с нужными названиями столбцов
date_sum = pd.DataFrame(results, columns=columns)
date_sum.head(15)

,date,job_industry_category,sum
0,2017-01-01,Argiculture,642.7000
1,2017-01-01,Entertainment,2472.5200
2,2017-01-01,Financial Services,9144.0300
3,2017-01-01,Health,9599.8100
4,2017-01-01,IT,2372.8700
5,2017-01-01,Manufacturing,9581.2500
6,2017-01-01,NaN,11736.4700
7,2017-01-01,Property,6228.2803
8,2017-01-01,Retail,5577.2100
9,2017-01-01,Telecommunications,2674.3100


3) Вывести количество онлайн-заказов для всех брендов в рамках подтвержденных заказов клиентов из сферы IT. — (1 балл)

In [29]:
# Выполняем SQL-запрос
cur.execute("""
            select  t.brand,
                    count(t.transaction_id) as count
            from homework_2.transaction as t
            left join homework_2.customer as c
            on t.customer_id = c.customer_id
            where t.order_status = 'Approved'
            and c.job_industry_category = 'IT'
            group by 1
            order by 2
            """)
conn.commit()
results = cur.fetchall()
# Получаем описание столбцов таблицы
columns = [desc[0] for desc in cur.description]
# Создаем датафрейм с нужными названиями столбцов
brands_count = pd.DataFrame(results, columns=columns)
brands_count.head(15)

,brand,count
0,NaN,16
1,OHM Cycles,156
2,Trek Bicycles,160
3,Norco Bicycles,163
4,Giant Bicycles,171
5,WeareA2B,190
6,Solex,220


4) Найти по всем клиентам сумму всех транзакций (list_price), максимум, минимум и количество транзакций, отсортировав результат по убыванию суммы транзакций и количества клиентов. Выполните двумя способами: используя только group by и используя только оконные функции. Сравните результат. — (2 балла)

In [33]:
# Выполняем SQL-запрос
cur.execute("""
            select  c.customer_id,
                    sum(t.list_price) as transaction_sum,
                    max(t.list_price) as max_list_price,
                    min(t.list_price) as min_list_price,
                    count(t.transaction_id) as transaction_count
            from homework_2.transaction as t
            left join homework_2.customer as c
            on t.customer_id = c.customer_id
            group by 1
            order by 2 desc, 5 desc
            """)
conn.commit()
results = cur.fetchall()
# Получаем описание столбцов таблицы
columns = [desc[0] for desc in cur.description]
# Создаем датафрейм с нужными названиями столбцов
brands_count = pd.DataFrame(results, columns=columns)
brands_count.head(15)

,customer_id,transaction_sum,max_list_price,min_list_price,transaction_count
0,2183.0,19071.322,2005.66,230.91,14
1,1129.0,18349.270,1992.93,290.62,13
2,1597.0,18052.680,2091.47,360.40,12
3,941.0,17898.459,2091.47,1057.51,10
4,2788.0,17258.940,2083.94,183.86,11
5,936.0,17160.240,2005.66,183.86,12
6,1887.0,17133.932,2091.47,688.63,11
7,1302.0,17035.830,1977.36,71.16,13
8,1140.0,16199.240,2083.94,183.86,13
9,2309.0,16122.341,2091.47,290.62,12


In [39]:
# Выполняем SQL-запрос
cur.execute("""
            select  c.customer_id,
                    SUM(t.list_price) OVER (PARTITION BY c.customer_id) AS total_sum,
                    MAX(t.list_price) OVER (PARTITION BY c.customer_id) AS max_transaction,
                    MIN(t.list_price) OVER (PARTITION BY c.customer_id) AS min_transaction,
                    COUNT(*) OVER (PARTITION BY c.customer_id) AS transaction_count
            from homework_2.transaction as t
            left join homework_2.customer as c
            on t.customer_id = c.customer_id
            order by 2 desc, 5 desc
            """)
conn.commit()
results = cur.fetchall()
# Получаем описание столбцов таблицы
columns = [desc[0] for desc in cur.description]
# Создаем датафрейм с нужными названиями столбцов
brands_count = pd.DataFrame(results, columns=columns)
brands_count.head(15)

,customer_id,total_sum,max_transaction,min_transaction,transaction_count
0,2183.0,19071.32,2005.66,230.91,14
1,2183.0,19071.32,2005.66,230.91,14
2,2183.0,19071.32,2005.66,230.91,14
3,2183.0,19071.32,2005.66,230.91,14
4,2183.0,19071.32,2005.66,230.91,14
5,2183.0,19071.32,2005.66,230.91,14
6,2183.0,19071.32,2005.66,230.91,14
7,2183.0,19071.32,2005.66,230.91,14
8,2183.0,19071.32,2005.66,230.91,14
9,2183.0,19071.32,2005.66,230.91,14


* Оконные функции не изменяют количество строк

5) Найти имена и фамилии клиентов с минимальной/максимальной суммой транзакций за весь период (сумма транзакций не может быть null). Напишите отдельные запросы для минимальной и максимальной суммы. — (2 балла)

In [42]:
# Выполняем SQL-запрос
cur.execute("""
            select  c.first_name,
                    c.last_name,
                    t.list_price
            from homework_2.transaction as t
            left join homework_2.customer as c
            on t.customer_id = c.customer_id
            where t.list_price in (select max(list_price) from homework_2.transaction)
            """)
conn.commit()
results = cur.fetchall()
# Получаем описание столбцов таблицы
columns = [desc[0] for desc in cur.description]
# Создаем датафрейм с нужными названиями столбцов
brands_count = pd.DataFrame(results, columns=columns)
brands_count

,first_name,last_name,list_price
0,Lauree,O'Donnell,2091.47
1,Merv,Barradell,2091.47
2,Wendel,Headland,2091.47
3,Dane,Swansbury,2091.47
4,Marcile,Christley,2091.47
...,...,...,...
460,Ronnica,Grebner,2091.47
461,Rod,Inder,2091.47
462,Jordanna,Skyme,2091.47
463,Johan,Dudderidge,2091.47


In [43]:
# Выполняем SQL-запрос
cur.execute("""
            select  c.first_name,
                    c.last_name,
                    t.list_price
            from homework_2.transaction as t
            left join homework_2.customer as c
            on t.customer_id = c.customer_id
            where t.list_price in (select min(list_price) from homework_2.transaction where list_price is not null)
            """)
conn.commit()
results = cur.fetchall()
# Получаем описание столбцов таблицы
columns = [desc[0] for desc in cur.description]
# Создаем датафрейм с нужными названиями столбцов
brands_count = pd.DataFrame(results, columns=columns)
brands_count

,first_name,last_name,list_price
0,Faythe,Rance,12.01
1,Annamarie,Bouzan,12.01
2,Ebba,Comport,12.01
3,Alexis,Aronov,12.01
4,Josias,Mulliner,12.01
...,...,...,...
190,Bradly,Crosse,12.01
191,Fabien,Iacapucci,12.01
192,Jermayne,Hryniewicki,12.01
193,Diane,Busk,12.01


6) Вывести только самые первые транзакции клиентов. Решить с помощью оконных функций. — (1 балл)

In [56]:
# Выполняем SQL-запрос
cur.execute("""
            select *
            from
            (select c.customer_id,
                    t.transaction_id,
                    t.transaction_date,
                    dense_rank() over(partition by c.customer_id order by t.transaction_date)
            from homework_2.transaction as t
            left join homework_2.customer as c
            on t.customer_id = c.customer_id) as new_table
            where new_table.dense_rank = 1
            """)
conn.commit()
results = cur.fetchall()
# Получаем описание столбцов таблицы
columns = [desc[0] for desc in cur.description]
# Создаем датафрейм с нужными названиями столбцов
brands_count = pd.DataFrame(results, columns=columns)
brands_count

,customer_id,transaction_id,transaction_date,dense_rank
0,1.0,9785,2017-01-05,1
1,2.0,2261,2017-05-04,1
2,3.0,10302,2017-02-23,1
3,4.0,12441,2017-04-03,1
4,5.0,2291,2017-03-03,1
...,...,...,...,...
3515,3497.0,8276,2017-09-01,1
3516,3498.0,13469,2017-02-03,1
3517,3499.0,2794,2017-01-12,1
3518,3500.0,6309,2017-01-09,1


7) Вывести имена, фамилии и профессии клиентов, между транзакциями которых был максимальный интервал (интервал вычисляется в днях) — (2 балла).

In [59]:
# Выполняем SQL-запрос
cur.execute("""
            with table_interval as (
            select  c.first_name,
                    c.last_name,
                    c.job_title,
                    t.transaction_date,
                    lag(t.transaction_date) over(partition by t.customer_id order by t.transaction_date),
                    (t.transaction_date - lag(t.transaction_date) over(partition by t.customer_id order by t.transaction_date)) as interval
            from homework_2.transaction as t
            left join homework_2.customer as c
            on t.customer_id = c.customer_id                
            )
            select first_name,
                   last_name,
                   job_title,
                   interval
            from table_interval
            where interval in (select max(interval) from table_interval)
            """)
conn.commit()
results = cur.fetchall()
# Получаем описание столбцов таблицы
columns = [desc[0] for desc in cur.description]
# Создаем датафрейм с нужными названиями столбцов
brands_count = pd.DataFrame(results, columns=columns)
brands_count

,first_name,last_name,job_title,interval
0,Susanetta,NaN,Legal Assistant,357
